# <u>Homework:</u> Solving the Knapsack Problem


Remark: Commenting your code will be part of the evaluation.



**Names** of each member of the group (4 members max), together with percentages of participation:

Melvin Gode (95%), Andrej Perković (5%)



## Description of the Context: Bounded Knapsack Problem

You are given a set of items, each with an associated value and weight, and a knapsack with a maximum weight capacity. Each item is available a certain number of times. The goal is to determine the most valuable combination of items to place in the knapsack without exceeding its weight capacity.

**Problem Parameters:**

- **Set of Items:** There is a finite set of items, often denoted as $I = \{1, 2, \ldots, n\}$.

- **Item Properties:**
  - Each item $i$ has an associated value $v_i$ (positive real number), representing the profit gained by including the item in the knapsack.
  - Each item $i$ has an associated weight $w_i$ (positive real number), indicating the weight of the item.
  - Each item $i$ has an associated maximum quantity $b_i$ (a non-negative integer), specifying the maximum number of times that item can be selected.

- **Knapsack Capacity:** The knapsack has a maximum weight capacity, denoted as $W$ (a positive real number).

**Objective:**

The objective of the bounded knapsack problem is to determine a selection of items, taking into account the quantity bounds, that maximizes the total value of the selected items while ensuring that the total weight does not exceed the knapsack's capacity.


In [ ]:
import numpy as np
import time as time

## I. Fractional Bounded Knapsack

In the fractional version of the problem, you are allowed to break items into bits as tiny as you wish for.

**Q1.** Write a function `knapfrac(values, weights, quantities, capacity)` whose arguments are:
 - a list of values (values of items),
 - a list of weights (weights of items),
 - and a list of bounds (upper bounds on the number of items of each type),
 - an integer capacity (maximum weight the knapsack can hold),

 and which returns the maximum value of a fractional knapsak.

In [ ]:
# Since we can select fractions of the objects, we can be sure that picking as much as possible of the most profitable object is the best decision at every step.
# This corresponds to a greedy algorithm.
# Thus, the way we will proceed is by sorting objects according to their profitability (value over weight) and take as much as possible of the best one at every iteration.

def knapfrac_value(values, weights, quantities, capacity):
  values=np.array(values)
  quantities=np.array(quantities)
  weights = np.array(weights)

  # In case of any funny business, we protect ourselves from special cases
  values[quantities<=0]=-1
  if sum(weights<=0) : return "Error : No negative weights allowed"

  weightsum = 0
  total_value = 0
  nb_picked = 0
  values_per_kg = np.argsort(values/weights) # We rank the objects by profitability

  # We select objects as long as we still have some capacity remaining in our bag (and objects to select)
  while(weightsum < capacity) and nb_picked<len(values):
    # First, we chose the object with the maximum value per weight among all available ones
    choice = values_per_kg[-nb_picked -1]
    if values[choice]<= 0 : break # If the best option has a non-positive value, there's no point in continuing

    # We take as much weight as we can of that object. Meaning the minimum between the remaining backpack capacity and the total amount of that object that is available (e.i. total weight available)
    selected_weight = min(capacity-weightsum, quantities[choice]*weights[choice])

    # We also convert that weight to a quantity by dividing it by the weight of one unit of that object
    selected_quantity =  selected_weight / weights[choice]

    total_value += values[choice] * selected_quantity
    weightsum += selected_weight

    # If we keep picking objects, this means we have taken all available quantities of the previous objects.
    # So the next object we will pick is the next most profitable one
    nb_picked += 1

  return total_value

**Q2.** Enhance the previous function so that it also outputs an associated solution of optimum value.

In [ ]:
def knapfrac_sol(values, weights, quantities, capacity):
  weightsum=0
  # The initial backpack is an empty array the ith spot in the array will correspond to the selected quantity of the ith object.
  backpack=np.zeros(len(values))

  # Transforming into numpy arrays for faster manipulations
  values = np.array(values)
  weights = np.array(weights)
  quantities = np.array(quantities)
  nb_picked = 0

  values[quantities<=0] =- 1
  if sum(weights<=0) : return "Error : No negative weights allowed"

  values_per_kg = np.argsort(values/weights)

  while(weightsum < capacity) and nb_picked<len(values):
    choice = values_per_kg[-nb_picked -1]
    if values[choice]<= 0 : break


    selected_weight = min(capacity - weightsum, quantities[choice]*weights[choice])
    selected_quantity =  selected_weight / weights[choice]

    backpack[choice] = selected_quantity # We add the selected quantity of the selected object to the spot corresponding to it in the backpack array
    weightsum += selected_weight
    nb_picked +=1

  return backpack, values @ backpack # The scalar product bewteen the backpack and the values vector gives us the total value of the backpack

**Q3.** Test your implementation with suitable choices of values, weights, quantities, and capacities to ensure that it works correctly.

In [ ]:
#Test n°1 : obvious solution
values = [1, 3, 10]
weights = [2, 2, 5]
quantities = [5, 2, 1]
capacity = 10

# We can clearly see that the best object to take is number 3 as it has a profitability of 2, followed by object number 2 with a score of 1.5 and last, object one with 0.5.
# Object 3's total available weight is 1*5 which is inferior to our capacity of 10. Our total value is 0+10 = 10 and remaining capacity, 10-5 = 5.
# So we take object 2 which total available weight is 2*2 < 5. We take all of it (value = 6). Our total value is 10+6 = 16 and remaining capacity, 5-4 = 1.
# Repeat for object one, this time we can only take one kilogram of it which gives us value 0.5.

# We should end with total value of 16.5 and selected quantities 0.5, 2 and 1 respectively.

print("Test number one :")
print(knapfrac_value(values, weights, quantities, capacity))
print(knapfrac_sol(values, weights, quantities, capacity))

Test number one :
16.5
(array([0.5, 2. , 1. ]), 16.5)


In [ ]:
#Test °2 : not enough objects

values = [1, 3, 10]
weights = [2, 2, 5]
quantities = [5, 2, 1]
capacity = 40

# Since 5*2 + 2*2 + 1*5 <= 40, we should be able to select all objects here.
# Our total value should then be 5*1 + 2*3 + 1*10 = 21. And our backpack vector should be equal to the quantities vecor (5,2,1).

print("Test number two :")
print(knapfrac_value(values, weights, quantities, capacity))
print(knapfrac_sol(values, weights, quantities, capacity))

Test number two :
21.0
(array([5., 2., 1.]), 21.0)


In [ ]:
#Test °3 : negative values

values = [1, 3, -10]
weights = [2, 2, 5]
quantities = [5, 2, 1]
capacity = 40

# Given that the weights and quantities haven't changed, we are still able to pick all objects.
# However, object 3 now has a negative value. We should therefore avoid selecting it.
# Our total value should then be 5*1 + 2*3 = 11. And our backpack vector should be equal to the quantities vector for positive value objects and 0 otherwise.

print("Test number three :")
print(knapfrac_value(values, weights, quantities, capacity))
print(knapfrac_sol(values, weights, quantities, capacity))

Test number three :
11.0
(array([5., 2., 0.]), 11.0)


In [ ]:
#Test °4 : a bit less obvious

values = [1, 3, -10, 60, 11, 4, 9]
weights = [2, 2, 5, 50, 9, 3, 11]
quantities = [5, 2, 1, 1, 2, 3, 2]
capacity = 60

# Here, it is a bit harder to directly see what the optimal solution is.
# Let's see what our functions output and the verify if it is feasible.

print("Test number four :")
print(knapfrac_value(values, weights, quantities, capacity))

backpack, value = knapfrac_sol(values, weights, quantities, capacity)
print((backpack, value))

# If the backpack vector is well feasible, the scalar product between it and the weights vector should be inferior or equal to our max capacity
print("Feasable solution :",backpack @ weights <= capacity)

Test number four :
74.8
(array([0.  , 2.  , 0.  , 0.58, 2.  , 3.  , 0.  ]), 74.8)
Feasable solution : True


# II. Bounded Knapsack Dynamically

**Q0.** Provide and explain a recurrence relation which the bounded knapsack problem satisfies.

##Answer:

Let $K(W, b, w, v)$ be the maximal value for a knapsack problem of capacity $W$, quantities $b$, weights $w$ and values $v$. Then this value is equal to the maximum value of a subproblem knapsack with *capacity* reduced by $w_i$ and the *quantity* $b_i$ reduced by one, to which we then add the value of object $i$.

$$K(W, b, w, v) = \max_i \{ K(W - w_i, b'_i, w, v) + v_i \}$$

With $b'_i := [b_1,b_2,...,b_i-1,...,b_{n-1},b_n]$

**Q1.** Write a function `knapdyn(values, weights, quantities, capacity)` which returns the maximum value of a bounded knapsack problem.

In [ ]:
def knapdyn_aux(values, weights, quantities, capacity) :
  #Base case, we don't have the capacity for any more objects. I.e. all the weights are superior to the capacity (for available objects)
  if sum(weights[quantities>0]>capacity)==len(weights[quantities>0]) :
    return 0

  # Otherwise there are still objects we can select
  max=0
  number_of_objects = len(values)

  # We are going to iterate over the objects to throw recursive calls and after that, select the best object out of them
  for choice in range(number_of_objects):

    # Obviously we don't even consider an object if it is heavier than we can afford of that there is no more of it left to take
    if quantities[choice]<= 0 or weights[choice]>capacity or values[choice]<=0: continue
    # Discarding an object if its value is negative is not mandatory but it will gain us some time. For example we go from a runtime of 11s to 0.2s in example 2.

    # For the recursive calls, we are going to need to reduce the quantity of the selected object by one, and reduce the capacity by the weight of the selected object
    quantities_temp = quantities.copy()
    quantities_temp[choice] -=1

    capacity_temp = capacity - weights[choice]

    # The total value for selecting a given object is equal to the value of that object plus the value of the associated sub-problem (i.e. recursive call)
    choice_value = values[choice] + knapdyn(values, weights, quantities_temp, capacity_temp)

    # Of course, we then output the highest value out of all encountered
    if choice_value > max:
      max = choice_value
  return max

def knapdyn(values, weights, quantities, capacity):
  values = np.array(values)
  weights=np.array(weights)
  quantities=np.array(quantities)

  if sum(weights <= 0) : return "Error : No negative weights allowed"

  return knapdyn_aux(values, weights, quantities, capacity)

**Q2.** Enhance `knapdyn` into an efficient function `knapdynMem` which uses memoization.

As said in question 0, the only changing informations over recursive calls are the remaining *capacity* and the remaining *quantities*. Therefore, these two informations perfectly allow us to differenciate between situations.

The *quantities* array gives us knowledge of what objects are still available.

It also indirectly gives us knowledge of how much capacity we have remaining because the units substracted to it over the course of the algorithm represent the selected objects (and from that we could deduce remaining capacity).

Therefore, knowing all the remaining quantities allows us to deduce all information needed to distinguish bewteen cases. So we are going to utilize the **quantities_temp** array as a key for memoization in a **dictionnary**.

(The quantities array wouldn't be enough because we need to account for the last selected object in the loop iteration as well).

In [ ]:
def knapdynMem_aux(values, weights, quantities, capacity, memory):
  if sum(weights[quantities>0]>capacity)==len(weights[quantities>0]) : return 0

  number_of_objects = len(values)
  max = 0
  for choice in range(number_of_objects):
    if quantities[choice]<= 0 or weights[choice]>capacity or values[choice]<=0: continue

    quantities_temp = quantities.copy()
    quantities_temp[choice]-=1
    capacity_temp = capacity - weights[choice]

    memory_id = hash(tuple(quantities_temp)) # We hash the quantities_temp array to create a unique ID for our dictionnary

    if memory_id not in memory : # If we haven't computed the solution for this configuration yet, store the value of the recursive call in the array
      memory[memory_id] = knapdynMem_aux(values, weights, quantities_temp, capacity_temp, memory) # We don't add values[choice] here for reasons explained just below

    # We do max value selection slightly different here :
    # Because we don't want the value of the subproblem to be depending on what object we selected last, we don't directly add the value to the recursive call but only use it in the max comparison.
    if memory[memory_id] + values[choice] > max :
      max = memory[memory_id] + values[choice]

  return max

def knapdynMem(values, weights, quantities, capacity):
  values = np.array(values)
  weights=np.array(weights)
  quantities=np.array(quantities)

  if sum(weights<=0) : return "Error : No negative weights allowed"

  memory = {}
  return knapdynMem_aux(values, weights, quantities, capacity, memory)


**Q3.** Modify the previous functions so that they also output an associated solution of optimum value.

We are now going to add a *backpack* argument to the `knapdyn` function, indicating us what objects have been selected up to now.

Because the backpack contains all the information we need to compute its value (with the *values* array), we do not need to keep track of the total value anymore, we can simply compute it from the backpack.

Of course, the backpack also gets modified for each recursive call by adding to it one unit of the selected object in the loop iteration.

In [ ]:
def knapdyn_sol_aux(values, weights, quantities, capacity, backpack): # The function now takes a backpack argument, indicating us what objects are selected up to now
  if sum(weights[quantities>0]>capacity) == len(weights[quantities>0]) :
    return backpack # The base case is still the same, only that we now return an unmodified backpack instead of the value of the sub-problem

  max=0
  best_backpack = backpack
  number_of_objects = len(values)
  for choice in range(number_of_objects):
    if quantities[choice]<= 0 or weights[choice]>capacity or values[choice]<=0: continue

    quantities_temp = quantities.copy()
    quantities_temp[choice] -=1
    capacity_temp = capacity - weights[choice]

    # We are now also going to need to pass the backpack to the recursive calls, but we also need to modify it according to the object we select in that loop iteration
    backpack_temp = backpack.copy()
    backpack_temp[choice] +=1

    choice_backpack = knapdyn_sol_aux(values, weights, quantities_temp, capacity_temp, backpack_temp)

    # The total value is now corresponding to the backpack we return. More precisely, it equals the scalar product between the backpack and the values array.
    choice_value = choice_backpack @ values
    if choice_value > max:
      max = choice_value
      best_backpack = choice_backpack

  return best_backpack

def knapdyn_sol(values, weights, quantities, capacity):
  values = np.array(values)
  weights=np.array(weights)
  quantities=np.array(quantities)

  if sum(weights<=0) : return "Error : No negative weights allowed"

  backpack=np.array([0]*len(values))
  backpack = knapdyn_sol_aux(values, weights, quantities, capacity, backpack)
  return backpack @ values, backpack

In [ ]:
def knapdynMem_sol_aux(values, weights, quantities, capacity, memory, backpack):
  if sum(weights[quantities>0]>capacity) == len(weights[quantities>0]) : return backpack

  number_of_objects = len(values)
  max= 0
  best_backpack = backpack
  for choice in range(number_of_objects):
    if quantities[choice]<= 0 or weights[choice]>capacity or values[choice]<=0: continue

    quantities_temp = quantities.copy()
    quantities_temp[choice]-=1
    capacity_temp = capacity - weights[choice]

    backpack_temp = backpack.copy()
    backpack_temp[choice] += 1

    memory_id = hash(tuple(quantities_temp))

    if memory_id not in memory :
      memory[memory_id] = knapdynMem_sol_aux(values, weights, quantities_temp, capacity_temp, memory, backpack_temp)

    choice_value = memory[memory_id] @ values
    if choice_value > max :
      max = choice_value
      best_backpack = memory[memory_id]

  return best_backpack

def knapdynMem_sol(values, weights, quantities, capacity):
  values = np.array(values)
  weights=np.array(weights)
  quantities=np.array(quantities)
  capacity=np.array(capacity)

  if sum(weights<=0) : return "Error : No negative weights allowed"

  backpack=np.array([0]*len(values))
  memory={}
  backpack = knapdynMem_sol_aux(values, weights, quantities, capacity, memory, backpack)
  return backpack @ values, backpack

**Q4.** Devise a suitable set of test instances, and use them compare the performances of your functions with timit.

In [ ]:
import pandas as pd
import timeit as tm
from IPython.display import display

#Test n°1 : obvious solution
v = [1, 3, 10]
w = [2, 2, 5]
q = [5, 2, 1]
c = 10

# Let's take our first easy example again.
# We can quite easily see that the optimal solution will be taking one unit of object 3 and two units of object 2, giving us a total value of 16.
# Let's see if that corresponds to the results we obtain

print("Test number one :")
data = {"Result" : [knapdyn(v,w,q,c), knapdyn_sol(v, w, q, c), knapdynMem(v,w,q,c), knapdynMem_sol(v,w,q,c)],
        "Time" : [tm.timeit("knapdyn(v,w,q,c)", globals=globals(), number=1),
                  tm.timeit("knapdyn_sol(v, w, q, c)", globals=globals(), number=1),
                  tm.timeit("knapdynMem(v,w,q,c)", globals=globals(), number=1),
                  tm.timeit("knapdynMem_sol(v,w,q,c)", globals=globals(), number=1)]}

time_comparison = pd.DataFrame(data, index=["knapdyn", "knapdyn_sol", "knapdynMem", "knapdynMem_sol"])
display(time_comparison)

Test number one :


Result      Time
knapdyn                      16  0.001700
knapdyn_sol     (16, [0, 2, 1])  0.007044
knapdynMem                   16  0.000645
knapdynMem_sol  (16, [0, 2, 1])  0.000812

In [ ]:
# Test n°2 : two different solutions

v = [4, 2, 2]
w = [3, 1, 1.5]
q = [1, 1, 2]
c = 4

# Now, it should be equivalent to take one unit of object one or two of object two.
# We have two solutions yeilding a value of 6 : (1,1,0) and (0,1,2).
# Let's see if the solutions are the same for knapdyn_sol and knapdynMem_sol

print('Test number two :')
data = {"Result" : [knapdyn(v,w,q,c), knapdyn_sol(v, w, q, c), knapdynMem(v,w,q,c), knapdynMem_sol(v,w,q,c)],
        "Time" : [tm.timeit("knapdyn(v,w,q,c)", globals=globals(), number=1),
                  tm.timeit("knapdyn_sol(v, w, q, c)", globals=globals(), number=1),
                  tm.timeit("knapdynMem(v,w,q,c)", globals=globals(), number=1),
                  tm.timeit("knapdynMem_sol(v,w,q,c)", globals=globals(), number=1)]}

time_comparison = pd.DataFrame(data, index=["knapdyn", "knapdyn_sol", "knapdynMem", "knapdynMem_sol"])
display(time_comparison)

Test number two :


Result      Time
knapdyn                      6  0.000230
knapdyn_sol     (6, [1, 1, 0])  0.000222
knapdynMem                   6  0.000136
knapdynMem_sol  (6, [1, 1, 0])  0.000174

Feasible solution : True


In [ ]:
# Test n°3

# Let's try a less obvious case. All our functions should output the same total value and same backpack for solution functions.

v = [0, -9, 7, 17, 7, 9, -6, 9, 2, 11]
w = [3, 8, 4, 11, 7, 8.5, 5, 8, 2, 12]
q = [2, 2, 2, 1, 1, 1, 2, 3, 4, 5]
c = 50
print('Test number three :')
data = {"Result" : [knapdyn(v,w,q,c), knapdyn_sol(v, w, q, c), knapdynMem(v,w,q,c), knapdynMem_sol(v,w,q,c)],
        "Time" : [tm.timeit("knapdyn(v,w,q,c)", globals=globals(), number=1),
                  tm.timeit("knapdyn_sol(v, w, q, c)", globals=globals(), number=1),
                  tm.timeit("knapdynMem(v,w,q,c)", globals=globals(), number=1),
                  tm.timeit("knapdynMem_sol(v,w,q,c)", globals=globals(), number=1)]}

time_comparison = pd.DataFrame(data, index=["knapdyn", "knapdyn_sol", "knapdynMem", "knapdynMem_sol"])
display(time_comparison)

# Let's now make sure that this solution is feasible
print("Feasible solution :",data["Result"][1][1] @ w <= c)

Test number two :


Result       Time
knapdyn                                           65  20.151599
knapdyn_sol     (65, [0, 0, 2, 1, 1, 0, 0, 3, 0, 0])  16.133342
knapdynMem                                        65   0.024427
knapdynMem_sol  (65, [0, 0, 2, 1, 1, 0, 0, 3, 0, 0])   0.030179

Feasible solution : True


As we could have expected, memoization functions are way more time-efficient than their no-memory counterparts.

It is also interesting to note though that `knapdyn_sol` is faster than `knapdyn`.

# III. Knapsack by Branch and Bound

Branch and bound is a general algorithmic technique used for solving optimization problems, including the knapsack problem. It systematically explores a search tree of potential solutions, pruning branches that cannot lead to an optimal solution.

The primary steps in the branch and bound approach are as follows:

 - **Branching:** The problem space is divided into smaller subproblems, typically through binary choices. For example, in the knapsack problem, each item can be either included (1) or excluded (0).

 - **Bounding:** At each node in the search tree, an upper bound on the optimal solution is computed. If the bound is worse than the current best solution found, that branch of the tree is pruned, as it cannot lead to an optimal solution.

 - **Backtracking:** The algorithm explores promising branches while keeping track of the best solution found so far. When it reaches a leaf node, it updates the best solution if a better one is found.

**Q1.** Which of the previous functions can be used to obtain an upper bound at each node?

**Answer:**
Since it is a constraint-relaxed version of the same problem, the *fractional knapsack* solution is always an upper-bound to a knapsack problem solution. (For the same parameters obviously).

We can therefore use `knapfrack` to upper-bound the solution we will find in any branch of the tree.

**Q2.** Write a function `knapBnB(values, weights, quantities, capacity)`  which returns the maximum value of a bounded knapsack problem with a branch and bound approach.

The approach we are going to take here is to examine the last object of the array at each iteration.

Then we have a binary choice : select one unit of it or stop considering this object.

We then throw recursive calls with different arguments depending on the option we selected.

<br>

Since we do that process object by object, it is important to note that there will be **no overlap** in the tree. For any single object, we either cut it off directly (select 0 units), or select one unit then cut it off (select one unit), ..., or select $q_i$ units of it and then have no other choice than to cut it off (select $q_i$ units).

Each of these options is different so we have no overlaping *inter-branches*. But `knapBnB` is also a recursive function so we apply the same process *intra branch* and thus have no overlap inside the same branch either.

This result is important because it means this approach is *divide and conquer* which means we don't need memoization for this recursive function.

In [ ]:
def branch(values, weights, quantities, capacity, lower_bound, current_value):

  upper_bound = current_value + knapfrac_value(values, weights, quantities, capacity)
  if upper_bound <= lower_bound : return lower_bound # Pruning the branch if it cannot give us a better solution

  # If we cannot pick up the last object of the array (either by weight or by quantity constraint), we delete it from the options available
  while (len(quantities) and (quantities[-1]<=0 or weights[-1]>capacity)):
    quantities = np.delete(quantities,-1)
    values = np.delete(values,-1)
    weights = np.delete(weights,-1)

  # After this procedure, we might have no objects left to select.
  # If it is the case then we are in a leaf so we return the max between the global lower bound and our current value (which is a lower bound too)
  if not len(quantities) : return max(lower_bound, current_value)

  #First case : we select one unit of the last object of the array, meaning we reduce its quantity by one and the capacity by its weight
  q1 = quantities.copy()
  q1[-1] -= 1
  capacity_temp = capacity - weights[-1]
  lower_bound = branch(values, weights, q1, capacity_temp, lower_bound, current_value + values[-1])
  # This case translates by adding the value of the object to our current value

  #Second case : we select none of the last object, so we delete it entirely from the options available
  if not len(quantities)-1 : return max(lower_bound, current_value) # We might have no more objects after the one we just decided to not select, meaning it is the end of this branch
  q2 = np.delete(quantities,-1)
  w2 = np.delete(weights,-1)
  v2 = np.delete(values,-1)
  lower_bound = branch(v2, w2, q2, capacity, lower_bound, current_value)

  # In order to maximize the utility of branching, it is important that we put the first case before the second one
  # as selecting as many objects as possible will help us reach a better lower bound faster than selecting none.
  # We are later going to see a way to further improve this.

  return lower_bound

def knapBnB(values, weights, quantities, capacity):
  values = np.array(values)
  weights=np.array(weights)
  quantities=np.array(quantities)

  values[quantities<=0]=-1
  if sum(weights<=0) : return "Error : No negative weights allowed"

  return branch(values, weights, quantities, capacity,0,0)

**Q3.** Test your implementation with suitable choices of values, weights, quantities, and capacities to ensure that it works correctly.

In [ ]:
n=10
W=20
np.random.seed(1)

number_of_tests = 1000

# Since we already ensured knapdynMem woked correctly, let's perform a series of small sized tests and compare the results of the two functions
# If the two functions ever output different values, we know something will be wrong

for i in range(number_of_tests):
  v = np.random.randint(0,100, n)
  w = np.random.randint(2, 110, size=n)
  q = np.random.randint(1,10, size=n)

  mem = knapdynMem(v, w, q, c)
  bnb = knapBnB(v, w, q, c)

  print("knapdynMem :",mem)
  print("knapBnB :",bnb,"\n")
  if bnb != mem:
    print("Error : different values !")
    break

  if i == number_of_tests-1:
    print("All values match :)")

**Q4** Using timit, compare the performances of `knapBnB` and `knapdynMem`.

In [ ]:
# Since we know knapdynMem is pretty efficient, let's try a test on a large sample

n = 90
np.random.seed(1)
v = np.random.randint(1,100, n)
w = np.random.randint(1, 100, size=n)
q = np.random.randint(1,4, size=n)
c = 70

data = {"Result" : [knapdynMem(v,w,q,c),
                    knapBnB(v,w,q,c)],
        "Time" : [tm.timeit("knapdynMem(v,w,q,c)", globals=globals(), number=1),
                  tm.timeit("knapBnB(v,w,q,c)", globals=globals(), number=1)]}

time_comparison = pd.DataFrame(data, index=["knapdynMem",
                                            "knapBnB"])

# This cell's total runtime should be around one minute

display(time_comparison)

Result        Time
knapdynMem     778  129.715330
knapBnB        778    0.038494

In [ ]:
# To make sure this result is not just chance, let's perform a larger battery of smaller sized tests

n = 50
number_of_tests = 100
c = 30
data = {"knapdynMem":[], "knapBnB":[]}

for i in range(number_of_tests):
  np.random.seed(i+1)
  v = np.random.randint(1,100, n)
  w = np.random.randint(1, 100, n)
  q = np.random.randint(1,4, n)

  data["knapdynMem"].append(tm.timeit("knapdynMem(v,w,q,c)", globals=globals(), number=1))
  data["knapBnB"].append(tm.timeit("knapBnB(v,w,q,c)", globals=globals(), number=1))

times = pd.DataFrame(data)

display(times)

print("knapdynMem average runtime :", np.mean(times['knapdynMem']))
print("knapBnB average runtime :", np.mean(times['knapBnB']),'\n')

print("knapBnB is on average", np.round(np.mean(times['knapdynMem'])/np.mean(times['knapBnB']),2),"times faster than knapdynMem")

print("knapBnB is faster than kanpdynMem", np.mean(data["knapBnB"] < data["knapdynMem"])*100,"% of the time")


knapdynMem   knapBnB
0     0.011183  0.005393
1     0.005033  0.005817
2     0.002128  0.003775
3     0.005688  0.007337
4     0.086813  0.023703
..         ...       ...
95    0.006659  0.015620
96    0.010133  0.014169
97    0.002068  0.001895
98    0.005372  0.006558
99    0.049517  0.011921

[100 rows x 2 columns]

knapdynMem average runtime : 0.034750113109994346
knapBnB average runtime : 0.008102825250000478 

knapBnB is on average 4.29 times faster than knapdynMem
knapBnB is faster than kanpdynMem 100.0 % of the time


###KnapBnB_heuristic

To go as fast as possible, we want to prune as many branches as possible.

To take advantage of that, we are sorting objects according to their value over weight ratio. This makes it so that the first objects we select are the most profitable and thus, we can imagine, more likely to be included in the optimal solution.

This will give us a faster approximation of the best lower bound, allowing us to prune many more other branches.

In [ ]:
def knapBnB_heuristic(values, weights, quantities, capacity):
  values = np.array(values)
  weights=np.array(weights)
  quantities=np.array(quantities)

  values[quantities<=0]=-1
  if sum(weights<=0) : return "Error : No negative weights allowed"

  values_over_weights = values/weights
  sort_indices = values_over_weights.argsort()
  values = values[sort_indices]
  weights = weights[sort_indices]
  quantities = quantities[sort_indices]

  return branch(values, weights, quantities, capacity,0,0)

Comparing performances of `knapBnB` and `knapBnB_heuristic`

In [ ]:
n = 1000
np.random.seed(1)

v = np.random.randint(0,100, n)
w = np.random.randint(2, 100, n)
q = np.random.randint(1,100, n)
c = 110

data = {"Result" : [knapBnB(v,w,q,c),
                    knapBnB_heuristic(v,w,q,c)],
        "Time" : [tm.timeit("knapBnB(v,w,q,c)", globals=globals(), number=1),
                  tm.timeit("knapBnB_heuristic(v,w,q,c)", globals=globals(), number=1)]}

time_comparison = pd.DataFrame(data, index=["knapBnB",
                                            "knapBnB_heuristic"])

# The total runtime for this cell should be around 2 minutes

display(time_comparison)

Result       Time
knapBnB              5335  69.657804
knapBnB_heuristic    5335   0.031777

We can see that, for 10 times the number of objects and 100 times the maximum quantities per object, `knapBnB_heuristic` is still even more efficient than `knapBnB`. This little tweak seems to be very efficient.

Let's run a similar battery of tests between our two functions.

In [ ]:
n = 150
number_of_tests = 100
c = 50
data = {"knapBnB":[], "knapBnB_heuristic":[]}

for i in range(number_of_tests):
  np.random.seed(i+1)
  v = np.random.randint(1,100, n)
  w = np.random.randint(1, 100, n)
  q = np.random.randint(1,10, n)

  data["knapBnB"].append(tm.timeit("knapBnB(v,w,q,c)", globals=globals(), number=1))
  data["knapBnB_heuristic"].append(tm.timeit("knapBnB_heuristic(v,w,q,c)", globals=globals(), number=1))

times = pd.DataFrame(data)

display(times)

print("knapBnB average runtime :", np.mean(times['knapBnB']))
print("knapBnB_heuristic average runtime :", np.mean(times['knapBnB_heuristic']),'\n')
print("knapBnB_heuristic is on average", round(np.mean(times['knapBnB'])/np.mean(times['knapBnB_heuristic']),2),"times faster than knapBnB")
print("knapBnB_heuristic is faster than knapBnB", np.mean(data["knapBnB_heuristic"] < data["knapBnB"])*100,"% of the time")

knapBnB  knapBnB_heuristic
0   0.081585           0.010670
1   0.264408           0.007157
2   0.169249           0.057599
3   0.572206           0.029874
4   1.161604           0.009502
..       ...                ...
95  0.201042           0.010588
96  0.393038           0.020603
97  0.240333           0.007994
98  0.566867           0.011573
99  0.170550           0.011714

[100 rows x 2 columns]

knapBnB average runtime : 0.28596704632001546
knapBnB_heuristic average runtime : 0.011592595850020189 

knapBnB_heuristic is on average 24.67 times faster than knapBnB
knapBnB_heuristic is faster than knapBnB 100.0 % of the time


# IV. Complexity analysis

With explanations, give the respective complexities of `knapfrac`, `knapdyn`, `knapdynMem`, and `knapBnB`.

Are the previous practical comparisons consistent with these theoretic results?

##Answer:
First let's define :

*   $n :=$ number of objects
*   $Q := \sum_i^n q_i$

<br>


**`knapfrac`** - In this algorithm, we have to sort the values/weights array. After that,  in the worst case go through the whole list while deciding which ones can fit in the knapsack. So our total complexity is $\mathcal{O}(n\cdot \log(n) + n) = \mathcal{O}(n\cdot \log(n))$

<br>


**`knapdyn`** - Each call makes at most $n$ recursive calls if all objects are available. We do this at most "total number of selectable objects" times which is exactly $Q$.

In every call, we also go through the whole list of $n$ objects.

So the complexity of knapdyn is $\mathcal{O}(n\cdot n^Q) = \mathcal{O}(n^Q)$ .

<br>

**`knapdynMem`** - In this algorithm, we explore at worst all possible combinations of objects. This is equivalent to all possible backpacks with the given quantities array. We have $n$ spots to fill which can take up to $q_{max}+1$ different values.
So the worst case complexity of `knapdynMem` is $\mathcal{O}((q_{max}+1)^n)$.

We chose to keep the +1 in the formula for the case where all quantities are exactly one. Which would look like the complexity is constant time if we took the formula litterally.

<br>

**`knapBnB`** - In this algorithm, we make binary choices. The number of binary choices we make for object number $i$ is at worst $q_i + 1$ if we never prune a branch. And we repeat that process for all objects. So the total number of binary choices made is at most $\sum_i q_i + 1= Q + n$.

But in every call of `knapBnB`, we call `knapfrac` which has a complexity of $\mathcal{O}(n\cdot \log(n))$.
Thus, the worst case complexity of knapBnB is $\mathcal{O}(n\cdot \log(n)\cdot2^{Q+n})$.

If we can reasonably consider that all quantities are at least one, then we have $Q \geq n$.

Therefore, we can summarise the complexity of `knapBnB` as $\mathcal{O}(2^{Q})$.

<br>

In theory, `knapBnB`'s complexity looks worse than `kanpdynMem`. However, in practice, we observe that `knapBnB` is much faster. That is because, as we said, worst case complexity accounts for no pruning while in reality, this helps speed up the process a lot.

Even more surprisingly, this is also the case for `knapBnB_heuristic` which still has the exact same complexity as `knapBnB` in the case of no pruning at all.

# V. Unbounded Knapsack

In the unbounded version, there are no limitations on the quantity of each item that can be taken.

**Q1.** How can the previous functions can be used to solve the unbounded knapsack version?


**Answer:** Even though the quantities are unbounded, we can construct an array of quantities using theoretical bounds and call any of the functions dealing with bounded knapsack problem. Theoretical bounds are determined by calculating the maximum number of units of each object that can fit in the knapsack of the given capacity:
$$q_i = \left\lfloor\frac{W}{w_i}\right\rfloor$$


Let's give an example using our most efficient function, `knapBnB_heuristic` :

In [ ]:
def unbounded_knap(values, weights, capacity):
  values = np.array(values)
  weights = np.array(weights)
  theoretical_quantities = np.floor(capacity/weights)

  return knapdynMem_sol(values, weights, theoretical_quantities, capacity)

v = [0, 1, 7, 17, 4.5, 9]
w = [3, 8, 5, 11, 7, 8.5]
c = 20

print(unbounded_knap(v, w, c))

# And let's compare it to using knapdynMem_sol directly for these quantities :
q = [1,2,2,1,1, 3]

print(knapdynMem_sol(v, w, q, c))

(28.0, array([0, 0, 4, 0, 0, 0]))
(26.0, array([0, 0, 0, 1, 0, 1]))


Since calculating theoretical quantities is just adding linear time to our function, the overall complexity stays the exact same, whatever function we chose to use.